In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/spaceship-titanic/sample_submission.csv
/kaggle/input/spaceship-titanic/train.csv
/kaggle/input/spaceship-titanic/test.csv


In [2]:
train = pd.read_csv("/kaggle/input/spaceship-titanic/train.csv")
test = pd.read_csv("/kaggle/input/spaceship-titanic/test.csv")
submission = pd.read_csv("/kaggle/input/spaceship-titanic/sample_submission.csv")

In [3]:
cols = ["PassengerId","Name","Transported"]
X_train = train.drop(columns = cols)
y_train = train["Transported"]

X_test = test.drop(columns = ["PassengerId","Name"])

In [4]:
fill_cols = ["HomePlanet", "CryoSleep", "Cabin", "Destination", 'VIP',
       'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
X_train[fill_cols]=X_train[fill_cols].fillna("Missing")
X_train["Age"] = X_train["Age"].fillna(X_train["Age"].mean())

In [5]:
X_test[fill_cols]=X_test[fill_cols].fillna("Missing")
X_test["Age"] = X_test["Age"].fillna(X_test["Age"].mean())

In [6]:
cat_cols = X_train.select_dtypes(include='object').columns.tolist()

X = pd.get_dummies(X_train, columns=cat_cols)
X_test = pd.get_dummies(X_test, columns=cat_cols)

In [7]:
X_test = X_test.reindex(columns=X.columns, fill_value=0)

In [8]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(
    n_estimators=100,  # 트리 개수
    max_depth=10,      # 트리 최대 깊이
    random_state=42    # 랜덤 고정 (결과 재현 가능하게)
)

In [9]:
model.fit(X,y_train)

RandomForestRegressor(max_depth=10, random_state=42)

In [10]:
prediction = model.predict(X_test)

In [11]:
preds = (prediction >= 0.5).astype(str)

In [12]:
submission["Transported"] = preds

In [13]:
submission.to_csv('submission.csv', index=False)